---
* *** 원본 소스는 홍콩과기대 김성훈 교수님의  [Github link](https://github.com/hunkim/DeepLearningZeroToAll)를 참조하세요 ***     
* *** Watson studio notebook 에서 작업 가능하도록 수정하였으며***
* *** 한글로 설명된 부분은 제가 스터디 하면서 이해한 내용을 추가한 것입니다. ***
---

# Lab 5 Logistic Regression Classifier
* ***lab 5-1 하고 동일하게 작동하지만, input file 을 다르게 가져갑니다.***
* ***File asset 으로 insert to code 하면  pandas code 로 생성되기 때문에,  numpy 로 전환하는 작업이 필요합니다.*** 

In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

In [3]:

import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_ace6f71a1b9946cbb684ca0d9e6c34c0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xBNCWwO4mooXZRHuqZ71nRmyIlQt8ca3ZWN8pOo56X69Dx',   ## masking, use your own
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_ace6f71a1b9946cbb684ca0d9e6c34c0.get_object(Bucket='tensorflowlabwithwatsonstudio-donotdelete-pr-neiwaip4a29fcg',Key='data-03-diabetes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()



,-0.294118,0.487437,0.180328,-0.292929,0,0.00149028,-0.53117,-0.0333333,0.1
0,-0.882353,-0.145729,0.081967,-0.414141,0.000000,-0.207153,-0.766866,-0.666667,1
1,-0.058824,0.839196,0.049180,0.000000,0.000000,-0.305514,-0.492741,-0.633333,0
2,-0.882353,-0.105528,0.081967,-0.535354,-0.777778,-0.162444,-0.923997,0.000000,1
3,0.000000,0.376884,-0.344262,-0.292929,-0.602837,0.284650,0.887276,-0.600000,0
4,-0.411765,0.165829,0.213115,0.000000,0.000000,-0.236960,-0.894962,-0.700000,1


* ***pandas 로 읽었기 때문에  numpy로 convert 해줘야 합니다.***

[pd to numpy reference link](https://stackoverflow.com/questions/40972147/convert-pandas-pd-to-numpy-array-and-back)

In [5]:
xy = df_data_1.values
#xy = np.loadtxt('data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

(758, 8) (758, 1)


In [6]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 8])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([8, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [7]:
# Launch graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(10001):
    cost_val, _ = sess.run([cost, train], feed_dict={X: x_data, Y: y_data})
    if step % 500 == 0:
        print(step, cost_val)

0 0.828086
500 0.715032
1000 0.669607
1500 0.634058
2000 0.606034
2500 0.583908
3000 0.566361
3500 0.552346
4000 0.541053
4500 0.531868
5000 0.524322
5500 0.518065
6000 0.51283
6500 0.508413
7000 0.504656
7500 0.501439
8000 0.498665
8500 0.496258
9000 0.494157
9500 0.492313
10000 0.490687


In [8]:
# Accuracy report
h, c, a = sess.run([hypothesis, predicted, accuracy],
                  feed_dict={X: x_data, Y: y_data})
print("\nHypothesis: ", h, "\nCorrect (Y): ", c, "\nAccuracy: ", a)


Hypothesis:  [[ 0.91558039]
 [ 0.22683522]
 [ 0.93606353]
 [ 0.33854681]
 [ 0.70947021]
 [ 0.94387567]
 [ 0.63418144]
 [ 0.26217601]
 [ 0.46605211]
 [ 0.647578  ]
 [ 0.20207596]
 [ 0.26094097]
 [ 0.35577044]
 [ 0.74865079]
 [ 0.48198602]
 [ 0.70029336]
 [ 0.91255689]
 [ 0.81196183]
 [ 0.5600341 ]
 [ 0.64852798]
 [ 0.1085592 ]
 [ 0.62158   ]
 [ 0.6815443 ]
 [ 0.38928407]
 [ 0.92176944]
 [ 0.51300806]
 [ 0.57887459]
 [ 0.72995311]
 [ 0.43030149]
 [ 0.94746941]
 [ 0.78300023]
 [ 0.57396239]
 [ 0.82060015]
 [ 0.37257335]
 [ 0.65679973]
 [ 0.83231843]
 [ 0.58754331]
 [ 0.4831382 ]
 [ 0.37859002]
 [ 0.77286875]
 [ 0.15560597]
 [ 0.4124487 ]
 [ 0.09964141]
 [ 0.60118902]
 [ 0.92161381]
 [ 0.75076461]
 [ 0.73070532]
 [ 0.90559077]
 [ 0.93368894]
 [ 0.91864413]
 [ 0.22811721]
 [ 0.40429467]
 [ 0.96732706]
 [ 0.24933477]
 [ 0.4874171 ]
 [ 0.165512  ]
 [ 0.74516839]
 [ 0.87808496]
 [ 0.50680178]
 [ 0.94101411]
 [ 0.70664757]
 [ 0.67294621]
 [ 0.834916  ]
 [ 0.57730496]
 [ 0.57168728]
 [ 0.942204

* ***accuracy 가 높지는 않네요***     
'''
0 0.82794
200 0.755181
400 0.726355
600 0.705179
800 0.686631
...
9600 0.492056
9800 0.491396
10000 0.490767

...

 [ 1.]
 [ 1.]
 [ 1.]]
Accuracy:  0.762846
'''